In [1]:
!nvidia-smi

Wed May 14 15:44:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.41                 Driver Version: 566.41         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX350         WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   61C    P8             N/A / 5001W |     997MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install langchain==0.1.14
!pip install langchain-experimental==0.0.56
!pip install langchain-community==0.0.31
!pip install faiss-cpu==1.8.0
!pip install pdfplumber==0.11.0
!pip install gradio==4.25.0

  Using cached gradio-4.25.0-py3-none-any.whl.metadata (15 kB)
  Using cached gradio_client-0.15.0-py3-none-any.whl.metadata (7.1 kB)
Using cached gradio-4.25.0-py3-none-any.whl (17.1 MB)
Using cached gradio_client-0.15.0-py3-none-any.whl (313 kB)

  Attempting uninstall: gradio-client

    Found existing installation: gradio_client 1.10.0

    Uninstalling gradio_client-1.10.0:

      Successfully uninstalled gradio_client-1.10.0

   ---------------------------------------- 0/2 [gradio-client]
  Attempting uninstall: gradio
   ---------------------------------------- 0/2 [gradio-client]
    Found existing installation: gradio 5.29.0
   ---------------------------------------- 0/2 [gradio-client]
   -------------------- ------------------- 1/2 [gradio]
   -------------------- ------------------- 1/2 [gradio]
   -------------------- ------------------- 1/2 [gradio]
   -------------------- ------------------- 1/2 [gradio]
   -------------------- ------------------- 1/2 [gradio]
   ------

In [3]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
!pip install serpapi


In [5]:
!pip install requests beautifulsoup4 markdownify tqdm

In [6]:
!pip install --upgrade gradio


  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
  Using cached gradio_client-1.10.0-py3-none-any.whl.metadata (7.1 kB)
Using cached gradio-5.29.0-py3-none-any.whl (54.1 MB)
Using cached gradio_client-1.10.0-py3-none-any.whl (322 kB)

  Attempting uninstall: gradio-client

    Found existing installation: gradio_client 0.15.0

    Uninstalling gradio_client-0.15.0:

      Successfully uninstalled gradio_client-0.15.0

   ---------------------------------------- 0/2 [gradio-client]
  Attempting uninstall: gradio
   ---------------------------------------- 0/2 [gradio-client]
    Found existing installation: gradio 4.25.0
   ---------------------------------------- 0/2 [gradio-client]
    Uninstalling gradio-4.25.0:
   ---------------------------------------- 0/2 [gradio-client]
   -------------------- ------------------- 1/2 [gradio]
   -------------------- ------------------- 1/2 [gradio]
      Successfully uninstalled gradio-4.25.0
   -------------------- ---------------

In [7]:
!pip install google-search-results

In [8]:
!pip show google-search-results


Name: google_search_results
Version: 2.4.2
Summary: Scrape and search localized results from Google, Bing, Baidu, Yahoo, Yandex, Ebay, Homedepot, youtube at scale using SerpApi.com
Home-page: https://github.com/serpapi/google-search-results-python
Author: vikoky
Author-email: victor@serpapi.com
License: MIT
Location: C:\Users\Radwa\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: requests
Required-by: 


In [9]:
!ollama pull llama3:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
llama3:8b                  365c0bd3c000    4.7 GB    Less than a second ago    
nomic-embed-text:latest    0a109f422b47    274 MB    19 seconds ago            


In [11]:
import gradio
print(gradio.__version__)

5.29.0


In [12]:
# --- Imports ---
import os
import re
import time
import traceback
import requests
import markdownify
import gradio as gr
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm # Use notebook version of tqdm for better display
from urllib.parse import urljoin, urlparse
from dotenv import load_dotenv # For API keys

# LangChain specific imports
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama # Assuming Ollama LLM
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import AgentType, initialize_agent 
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.tools import PythonREPLTool # Use experimental REPL tool

# --- Configuration ---
print("Loading environment variables...")
load_dotenv() # Load environment variables from .env file (if it exists)

# Model Configuration
EMBEDDING_MODEL = "nomic-embed-text" 
LLM_MODEL = "llama3:8b" 
print(f"Using Embedding Model: {EMBEDDING_MODEL}")
print(f"Using LLM Model: {LLM_MODEL}")

# Directory/Path Configuration
DOCS_DIR = "langchain_docs"
FAISS_INDEX_PATH = f"faiss_{DOCS_DIR}_{EMBEDDING_MODEL.replace('/', '_').replace(':','_')}_index" # Dynamic index path
print(f"Document Directory: {DOCS_DIR}")
print(f"FAISS Index Path: {FAISS_INDEX_PATH}")

# API Key Configuration
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY", "e3a57f45c13ba53fa20085d04fe48db3ca8a597a260087dfc5cc60651808169e")
if SERPAPI_API_KEY :
    print("SERPAPI_API_KEY found.")
else:
    print("SERPAPI_API_KEY not found or is placeholder. Web Search tool will be disabled.")


Loading environment variables...
Using Embedding Model: nomic-embed-text
Using LLM Model: llama3:8b
Document Directory: langchain_docs
FAISS Index Path: faiss_langchain_docs_nomic-embed-text_index
SERPAPI_API_KEY found.


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import markdownify
from tqdm import tqdm
import time
import traceback
from urllib.parse import urljoin, urlparse
import re


# --- Web Scraping Function ---
def scrape_langchain_docs(output_dir="langchain_docs"):
    base_url = "https://python.langchain.com"
    initial_seed_url = "https://python.langchain.com/docs/get_started/introduction"
    max_pages_to_scrape = 100  # <<< SET YOUR LIMIT FOR QUICK TESTING

    os.makedirs(output_dir, exist_ok=True)
    urls_to_visit = {initial_seed_url}
    visited_urls = set()
    scraped_content_urls = set()
    session = requests.Session()
    print(f"Starting crawl from: {initial_seed_url}")
    print(f"Will stop after scraping a maximum of {max_pages_to_scrape} pages.")

    with tqdm(desc="Scraping documentation", unit="page") as pbar:
        while urls_to_visit:
            if len(scraped_content_urls) >= max_pages_to_scrape: # Check before popping
                print(f"\nReached max {max_pages_to_scrape} scraped pages. Finalizing.")
                break

            current_url = urls_to_visit.pop()
            if current_url in visited_urls:
                continue
            
            pbar.set_postfix_str(f"Scraped: {len(scraped_content_urls)}/{max_pages_to_scrape}, Queue: {len(urls_to_visit)}, Visiting: {current_url[:50]}...")
            visited_urls.add(current_url)

            try:
                response = session.get(current_url, timeout=15)
                response.raise_for_status()
                final_url = response.url

                if final_url in scraped_content_urls or (final_url in visited_urls and final_url != current_url):
                    if final_url != current_url: visited_urls.add(current_url)
                    if final_url not in scraped_content_urls:
                        soup_for_links = BeautifulSoup(response.text, 'html.parser')
                    else:
                        pbar.update(1)
                        time.sleep(0.01) # Very short sleep for already processed
                        continue

                soup = BeautifulSoup(response.text, 'html.parser')
                # ... (link discovery logic remains the same) ...
                new_links_found = 0
                for link_tag in soup.find_all('a', href=True):
                    href = link_tag['href']
                    next_url = urljoin(final_url, href)
                    next_url_parsed = urlparse(next_url)
                    next_url_no_fragment = next_url_parsed._replace(fragment="").geturl()

                    if next_url_no_fragment.startswith(base_url) and \
                       ("docs" in next_url_no_fragment or "/latest/" in next_url_no_fragment or re.search(r'/v\d+\.\d+/', next_url_no_fragment)) and \
                       next_url_no_fragment not in visited_urls and \
                       next_url_no_fragment not in urls_to_visit and \
                       "api.python.langchain.com" not in next_url_no_fragment:
                        urls_to_visit.add(next_url_no_fragment)
                        new_links_found +=1


                if final_url not in scraped_content_urls:
                    main_content = (
                        soup.find('article') or soup.find('main') or
                        soup.find('div', role='main') or
                        soup.find('div', class_='theme-doc-markdown markdown')
                    )
                    if main_content:
                        md_content = markdownify.markdownify(str(main_content), heading_style="ATX")
                        if not md_content.strip():
                            scraped_content_urls.add(final_url) 
                            pbar.update(1)
                            time.sleep(0.01)
                            continue

                        url_path = urlparse(final_url).path
                        name_parts = [part for part in url_path.split('/') if part]
                        filename_base = "_".join(name_parts)
                        if not filename_base: filename_base = "root_doc_page"
                        filename = re.sub(r'[<>:"/\\|?*]', '_', filename_base)[:2000] + ".md"
                        filepath = os.path.join(output_dir, filename)

                        with open(filepath, 'w', encoding='utf-8') as f:
                            f.write(f"# Source: {final_url}\n\n")
                            f.write(md_content)
                        scraped_content_urls.add(final_url) # Add to set *after* successful save

                        if len(scraped_content_urls) >= max_pages_to_scrape: # Check again after scraping
                            print(f"\nReached max {max_pages_to_scrape} scraped pages during content extraction. Finalizing.")
                            pbar.update(1) # Update for this last page
                            break # Break from the while loop

                pbar.update(1)
                time.sleep(0.1) # Keep this for politeness during actual requests

            except requests.exceptions.HTTPError as e:
                pass # Or log to a file
            except requests.exceptions.RequestException as e:
                print(f"Request error for {current_url}: {e}")
            except Exception as e:
                print(f"Unexpected error processing {current_url}: {e}")
                traceback.print_exc()
            
            if len(urls_to_visit) == 0 and len(scraped_content_urls) < max_pages_to_scrape:
                pbar.total = len(visited_urls) # Update total if queue is empty but limit not reached
            elif len(scraped_content_urls) < max_pages_to_scrape :
                 pbar.total = len(visited_urls) + len(urls_to_visit)
            else: # Limit reached
                pbar.total = len(visited_urls)


    print(f"\n✅ Scraper finished. Scraped content from {len(scraped_content_urls)} pages into '{output_dir}'. Visited {len(visited_urls)} URLs.")
scrape_langchain_docs()

Starting crawl from: https://python.langchain.com/docs/get_started/introduction
Will stop after scraping a maximum of 100 pages.


Scraping documentation:   4%|▍         | 100/2588 [00:43<18:01,  2.30page/s, Scraped: 99/100, Queue: 2487, Visiting: https://python.langchain.com/v0.2/docs/how_to/docu...]


Reached max 100 scraped pages during content extraction. Finalizing.

✅ Scraper finished. Scraped content from 100 pages into 'langchain_docs'. Visited 101 URLs.


In [20]:
DOCS_DIR="langchain_docs"
# Cell 3: Document Loading Function Definition
def safe_load_documents(docs_path=DOCS_DIR):
    """Loads markdown documents safely from the specified directory."""
    print(f"\n--- Loading Documents ---")
    print(f"Source Directory: {docs_path}")
    if not os.path.exists(docs_path) or not os.listdir(docs_path):
        print(f"WARNING: Directory '{docs_path}' is empty or does not exist. No documents to load.")
        return []

    # Use TextLoader for basic Markdown loading
    loader = DirectoryLoader(
        docs_path,
        glob="**/*.md",
        loader_cls=TextLoader,
        loader_kwargs={'encoding': 'utf-8', 'autodetect_encoding': True},
        show_progress=True,
        use_multithreading=True,
        silent_errors=True # Suppress errors during loading individual files
    )

    documents = []
    print(f"Attempting to load documents...")
    try:
        # Use tqdm.notebook.tqdm for progress bar in notebook
        loaded_items = loader.load() # Returns list of Document objects

        if not loaded_items:
            print("loader.load() returned empty list. No documents found or loaded.")
            return []

        print(f"loader.load() returned {len(loaded_items)} potential document(s). Validating...")
        invalid_count = 0
        empty_count = 0
        for i, doc in enumerate(loaded_items):
            # Basic validation
            if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
                if doc.page_content and doc.page_content.strip():
                    # Extract source URL added by scraper
                    source_match = re.match(r"# Source: (https?://[^\s]+)", doc.page_content)
                    if source_match:
                         doc.metadata['source'] = source_match.group(1)
                         doc.page_content = re.sub(r"# Source: .*\n\n?", "", doc.page_content, count=1) # Remove source line
                    elif 'source' not in doc.metadata: # Fallback
                        doc.metadata['source'] = doc.metadata.get('source', 'Unknown')

                    documents.append(doc)
                else:
                    empty_count += 1
            else:
                 # print(f"  WARNING: Item {i} (type: {type(doc)}) skipped - not valid Document structure.") # Verbose
                 invalid_count += 1

        print(f"Validation complete: Loaded {len(documents)} valid documents.")
        if empty_count > 0: print(f"Skipped {empty_count} documents with empty content.")
        if invalid_count > 0: print(f"Skipped {invalid_count} invalid items.")

    except Exception as e:
        print(f"ERROR during document loading process:")
        traceback.print_exc()
        return [] # Return empty list on error

    print(f"--- Finished Loading Documents ({len(documents)} loaded) ---")
    return documents

print("safe_load_documents function defined.")

safe_load_documents function defined.


In [21]:

# Define FAISS index path
FAISS_INDEX_PATH = "faiss_langchain_docs_nomic_index"
# Cell 4: RAG Setup Function Definition
def setup_document_rag(docs_path=DOCS_DIR, index_path=FAISS_INDEX_PATH, embedding_service=None):
    """Sets up RAG using FAISS. Loads existing index or builds a new one."""
    print(f"\n--- Setting up Document RAG ---")
    print(f"Document Source: {docs_path}")
    print(f"FAISS Index Path: {index_path}")

    if not embedding_service:
        print("ERROR: Embedding service (e.g., OllamaEmbeddings) is required for RAG setup.")
        return None

    # Check if index exists and can be loaded
    if os.path.exists(index_path):
        try:
            print(f"Attempting to load existing FAISS index from: {index_path}")
            vectorstore = FAISS.load_local(index_path, embedding_service, allow_dangerous_deserialization=True)
            print("FAISS index loaded successfully.")
            print(f"--- Document RAG Setup Complete (Loaded Existing Index) ---")
            return vectorstore.as_retriever(search_kwargs={'k': 5}) # Return retriever, retrieve top 5
        except Exception as e:
            print(f"WARNING: Could not load existing FAISS index (path: {index_path}): {e}. Will attempt to rebuild.")
           

    # Build new index
    print("Building new FAISS index...")
    documents = safe_load_documents(docs_path) # Load documents
    if not documents:
        print("ERROR: No valid documents loaded. Cannot build RAG index.")
        print(f"--- Document RAG Setup Failed (No Documents) ---")
        return None

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150) # Adjusted chunk size/overlap
    try:
        print("Splitting documents into chunks...")
        texts = text_splitter.split_documents(documents)
        print(f"Split {len(documents)} documents into {len(texts)} chunks.")
        if not texts:
            print("ERROR: No text chunks generated after splitting. Cannot create vector store.")
            print(f"--- Document RAG Setup Failed (No Chunks) ---")
            return None
    except Exception as e:
        print(f"ERROR: Failed during document splitting:")
        traceback.print_exc()
        print(f"--- Document RAG Setup Failed (Splitting Error) ---")
        return None

    # Create and save FAISS vector store
    try:
        print(f"Creating FAISS vector store from {len(texts)} chunks using '{EMBEDDING_MODEL}'...")
        vectorstore = FAISS.from_documents(texts, embedding_service)
        print("FAISS vector store created successfully.")
        print(f"Saving FAISS index to: {index_path}")
        vectorstore.save_local(index_path)
        print("FAISS index saved.")
        print(f"--- Document RAG Setup Complete (Built New Index) ---")
        return vectorstore.as_retriever(search_kwargs={'k': 5}) # Return retriever, retrieve top 5
    except Exception as e:
        print(f"ERROR: Failed to create or save FAISS vector store:")
        traceback.print_exc()
        print(f"--- Document RAG Setup Failed (Vector Store Creation/Save Error) ---")
        return None

print("setup_document_rag function defined.")

setup_document_rag function defined.


In [22]:
# Cell 5: Tools Setup Function Definition
def setup_tools(retriever=None, serpapi_key=None):
    """Initializes and returns a list of agent tools with refined descriptions."""
    print(f"\n--- Setting up Agent Tools ---")
    local_tools = [] # Use a local list

    # Tool 1: LangChain Document Search (RAG)
    if retriever:
        def doc_retriever_wrapper(query: str) -> str:
            """Searches the local LangChain documentation index."""
            print(f"  [Tool Call] LangChain Document Search: '{query[:50]}...'")
            try:
                docs = retriever.get_relevant_documents(query)
                if not docs:
                    print("    -> No relevant documents found.")
                    return "No relevant documents found in the local LangChain knowledge base for this specific query. If the question is *not* specifically about LangChain implementation, consider using Web Search."
                # Format the output
                formatted_docs = []
                for i, d in enumerate(docs):
                    source = d.metadata.get('source', 'Unknown Source')
                    content_preview = (d.page_content[:500] + '...') if len(d.page_content) > 500 else d.page_content
                    formatted_docs.append(f"Result {i+1}:\nSource: {source}\nContent Snippet:\n{content_preview}")
                print(f"    -> Found {len(docs)} relevant documents.")
                return "Relevant excerpts from LangChain Documentation:\n\n---\n\n".join(formatted_docs)
            except Exception as e:
                print(f"    -> Error in doc_retriever_wrapper: {e}")
                # traceback.print_exc() # Uncomment for debug
                return "Error occurred while searching the local knowledge base."

        local_tools.append(Tool(
            name="LangChain Document Search",
            func=doc_retriever_wrapper,
            description=(
                "Use this tool FIRST for questions specifically about LangChain features, concepts, implementation, classes, functions, or how-to examples (e.g., 'How to use LCEL?', 'What is LangGraph?', 'TextLoader example', 'Create a custom agent'). "
                "Input should be a specific question about LangChain. This tool searches the indexed LangChain documentation. Only use other tools if this one explicitly finds nothing relevant for a LangChain-specific question."
            )
        ))
        print("  Tool Added: LangChain Document Search")
    else:
        print("  INFO: Document retriever not available. 'LangChain Document Search' tool skipped.")

    # Tool 2: Web Search (SerpAPI)
    if serpapi_key :
        try:
            serpapi_search = SerpAPIWrapper(serpapi_api_key=serpapi_key)
            def web_search_wrapper(query: str) -> str:
                 """Performs a web search using SerpAPI."""
                 print(f"  [Tool Call] Web Search: '{query[:50]}...'")
                 try:
                     result = serpapi_search.run(query)
                     print(f"    -> Web search result received.")
                     return result
                 except Exception as e:
                     print(f"    -> Error during SerpAPI call: {e}")
                     return f"Error during web search: {e}"

            local_tools.append(Tool(
                name="Web Search",
                func=web_search_wrapper,
                description=(
                    "Use this tool for general knowledge questions, current events, real-time information, or topics NOT specific to the LangChain library implementation *after* trying 'LangChain Document Search' if the query seemed LangChain related but yielded no results. "
                    "Useful for questions about Python libraries other than LangChain, external services, or very recent developments. Input should be a concise search query."
                )
            ))
            print("  Tool Added: Web Search (SerpAPI)")
        except ImportError:
            print("  WARNING: 'google-search-results' package not installed. 'Web Search' tool unavailable. Install with: pip install google-search-results")
        except Exception as e:
            print(f"  WARNING: Failed to initialize SerpAPI tool: {e}. 'Web Search' tool unavailable.")
    elif not serpapi_key:
        print("  INFO: SERPAPI_API_KEY not found. 'Web Search' tool unavailable.")
    else: # Key is the placeholder
         print(f"  INFO: SERPAPI_API_KEY is placeholder. 'Web Search' tool unavailable.")

    # Tool 3: Python REPL
    try:
        python_repl = PythonREPLTool()
        def python_repl_wrapper(code: str) -> str:
             """Executes Python code using a REPL tool."""
             print(f"  [Tool Call] Python REPL: '{code[:50]}...'")
             try:
                 # Ensure input is treated as code, not description
                 # The PythonREPLTool expects a string of Python code
                 result = python_repl.run(code) # Pass code directly
                 print(f"    -> REPL execution finished.")
                 return result
             except Exception as e:
                 print(f"    -> Error during Python REPL execution: {e}")
                 return f"Error executing Python code: {e}"

        local_tools.append(Tool(
            name="Python REPL",
            func=python_repl_wrapper,
            description=(
                "Use this tool ONLY to execute specific, self-contained Python code snippets. Useful for calculations, simple standard library checks, or demonstrating basic Python syntax. "
                "Input MUST be valid Python code (e.g., 'print(2+2)', 'import math; math.sqrt(16)'). Do NOT use it to ask questions, search the web, interact with LangChain objects (unless the code defines them fully), or run complex scripts. Avoid using if the task can be done by other tools."
            )
        ))
        print("  Tool Added: Python REPL")
    except ImportError:
         print("  WARNING: 'langchain_experimental' or 'python-repl' might not be installed. 'Python REPL' tool unavailable. Install with: pip install langchain_experimental")
    except Exception as e:
        print(f"  WARNING: Failed to initialize Python REPL tool: {e}. 'Python REPL' tool unavailable.")

    if not local_tools:
        print("WARNING: No tools were successfully initialized!")
    else:
        print(f"Initialized Tools: {[tool.name for tool in local_tools]}")
    print(f"--- Finished Setting up Agent Tools ---")
    return local_tools

print("setup_tools function defined.")

setup_tools function defined.


In [23]:
# Cell 6: Agent Initialization Function Definition

# IMPORT THE NEW MEMORY TYPE
from langchain.memory import ConversationBufferMemory, ConversationTokenBufferMemory

def initialize_conversational_agent(tools_list, agent_llm):
    """
    Initializes and returns the conversational agent executor using the (deprecated)
    initialize_agent function with CHAT_CONVERSATIONAL_REACT_DESCRIPTION.
    Uses ConversationTokenBufferMemory to limit context size.
    """
    print(f"\n--- Initializing Conversational Agent ---")

    if not agent_llm:
        print("ERROR: LLM must be initialized before creating the agent.")
        print(f"--- Agent Initialization Failed (No LLM) ---")
        return None
    if not tools_list:
        print("WARNING: No tools provided to the agent. It will rely solely on its internal knowledge and memory.")

    # Setup memory with token limiting
    # Use a reasonable limit (e.g., 3500 tokens) for history to avoid exceeding LLM context
    # Pass the llm to the memory for accurate token counting
    MAX_MEMORY_TOKENS = 3500
    print(f"Initializing ConversationTokenBufferMemory (max_token_limit={MAX_MEMORY_TOKENS})...")
    try:
        memory = ConversationTokenBufferMemory(
            llm=agent_llm, # Pass LLM for token counting
            memory_key="chat_history",
            return_messages=True,
            output_key='output',
            max_token_limit=MAX_MEMORY_TOKENS
        )
        print("Memory initialized.")
    except Exception as e:
         print(f"ERROR initializing ConversationTokenBufferMemory: {e}")
         print("Falling back to ConversationBufferMemory (may cause context length issues).")
         # Fallback (less ideal)
         memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key='output')


    try:
        print(f"Initializing agent with AgentType: {AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION} (using deprecated initialize_agent)")
        executor = initialize_agent(
            tools=tools_list,
            llm=agent_llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=memory, # Use the token buffer memory
            verbose=True,
            handle_parsing_errors=( # Provide specific guidance on parsing errors
                "Parsing Error: Ensure your output is either a valid JSON blob with 'action' and 'action_input' keys, "
                "or the final answer prefixed with 'Final Answer:'. Check your thought process and generated output format carefully."
            ),
            max_iterations=13,
            early_stopping_method="generate",
        )
        print("Conversational agent initialized successfully.")
        print(f"--- Agent Initialization Complete ---")
        return executor
    except Exception as e:
        print(f"ERROR: Failed to initialize conversational agent using initialize_agent.")
        print(f"Error details: {e}")
        traceback.print_exc()
        print(f"--- Agent Initialization Failed ---")
        return None

print("initialize_conversational_agent function defined.")

initialize_conversational_agent function defined.


In [24]:
# Cell 7: Gradio Interface Setup Function Definition
def setup_gradio_interface(agent_executor_instance, agent_tools_list=None):
    """Sets up and returns the Gradio Blocks interface with a more user-friendly design."""
    print(f"\n--- Setting up Gradio Interface ---")

    if not agent_executor_instance:
        print("ERROR: Agent executor instance is None. Cannot setup Gradio interface.")
        print(f"--- Gradio Setup Failed ---")
        with gr.Blocks(theme=gr.themes.Default(primary_hue=gr.themes.colors.red), title="Chatbot Error") as demo_error:
            gr.Markdown("# Chatbot Initialization Error\nThe agent executor could not be initialized. Please check the logs in the previous cells for errors (e.g., LLM/Tool setup failed).")
        return demo_error

    failure_indicators = [
        "i cannot answer", "i can't answer", "i do not know", "i don't know",
        "i don't have that information", "no relevant documents found", "i couldn't find any information",
        "unable to find relevant information", "could not find information", "my knowledge is limited",
        "the provided context does not contain", "i am unable to provide an answer",
        "the search results do not provide", "i apologize, i cannot provide", "based on the search results",
        "i'm sorry, but i couldn't find", "the document search did not return relevant results",
        "the web search did not return relevant results", "no information found", "i'm afraid i can't",
        "i am not able to provide",
    ]
    failure_indicators += [f.replace("i ", "i'm ") for f in failure_indicators if "i " in f]
    failure_indicators += [f.replace("do not", "don't") for f in failure_indicators if "do not" in f]
    failure_indicators += [f.replace("can not", "can't") for f in failure_indicators if "can not" in f]
    failure_indicators = list(set(failure_indicators)) # Unique

    default_empty_answer = "I apologize, but I couldn't find a specific answer to your query using my available tools and knowledge at this time. Please try rephrasing or ask something else."

    def respond(message: str, chat_history_pairs: list[list[str | None]] | None):
        """
        Callback function for Gradio Blocks interface.
        Handles user input, invokes the agent, and formats the response.
        chat_history_pairs is a list of [user_msg, assistant_msg] pairs.
        """
        print(f"\n[Gradio] User Input: '{message}'")
        current_history_for_logic = chat_history_pairs or [] # Use a local var for logic

        if not message or message.strip() == "":
            print("[Gradio] Empty input received.")
            # Return 3 values: empty textbox, unchanged history for chatbot, unchanged history for state
            return "", current_history_for_logic, current_history_for_logic


        final_response_text = default_empty_answer
        try:
            print("[Gradio] Invoking agent executor...")
            # Conversational Agent with ConversationBufferMemory handles history internally.
            # We only provide the current user 'input'.
            langchain_response = agent_executor_instance.invoke({"input": message})

            response_text = langchain_response.get('output', "").strip() if isinstance(langchain_response, dict) else str(langchain_response).strip()
            print(f"[Gradio] Agent Raw Output (first 2000 chars): {response_text[:2000]}...")

            if not response_text:
                print("[Gradio] Agent returned an empty response.")
                is_failure = True
            else:
                response_lower = response_text.lower()
                is_failure = any(indicator in response_lower for indicator in failure_indicators)
                if is_failure:
                    # Check if the response is very short and contains a failure indicator
                    if len(response_text) < 100 and any(indicator in response_lower for indicator in failure_indicators):
                         print(f"[Gradio] Detected clear failure indicator in a short response.")
                    else: # Could be a valid response that happens to contain an indicator substring
                         is_failure = False # Re-evaluate, maybe it's not a true failure
                         print(f"[Gradio] Potential failure indicator found, but response is longer. Assuming valid unless empty.")
                         if not response_text.strip(): is_failure = True # Treat truly empty as failure

            if not is_failure and response_text.strip(): # Ensure it's not a failure AND not just whitespace
                final_response_text = response_text
                print("[Gradio] Agent response seems valid.")
            else:
                print("[Gradio] Agent response indicated failure, was empty, or was just whitespace. Using default failure message.")
                # final_response_text remains default_empty_answer

        except Exception as e:
            print(f"[Gradio] ERROR during agent.invoke:")
            traceback.print_exc()
            # Ensure final_response_text is always a string, even on error
            final_response_text = f"An error occurred during processing. Please check the notebook logs. Error type: {type(e).__name__}"


        # Append the new turn to the history that will be returned to Gradio
        current_history_for_logic.append([message, final_response_text])
        print(f"[Gradio] Responding to UI...")
        # Return 3 values:
        # 1. Empty string for the msg_textbox
        # 2. The updated current_history for the chatbot_display
        # 3. The updated current_history for the chat_history_state
        return "", current_history_for_logic, current_history_for_logic

    print("Setting up Gradio UI using gr.Blocks for more control...")
    # These globals are assumed to be defined in Cell 1
    global LLM_MODEL, EMBEDDING_MODEL
    tool_names = [t.name for t in agent_tools_list] if agent_tools_list else ["None"]
    tools_str = ', '.join(tool_names)

    custom_theme = gr.themes.Soft(
        primary_hue=gr.themes.colors.blue,
        secondary_hue=gr.themes.colors.sky,
        neutral_hue=gr.themes.colors.slate, # Using slate for a slightly more modern neutral
        font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
    ).set(
        body_background_fill="#f0f4f8", # Light blue-gray background
        block_background_fill="white",
        block_border_width="1px",
        block_shadow="*shadow_drop_lg",
        button_primary_background_fill="*primary_500",
        button_primary_background_fill_hover="*primary_600",
        button_primary_text_color="white",
        button_secondary_background_fill="*neutral_200",
        button_secondary_background_fill_hover="*neutral_300",
        button_secondary_text_color="*neutral_700",
    )

    with gr.Blocks(theme=custom_theme, title="LangChain AI Assistant", css=".gradio-container {max-width: 800px !important; margin: auto !important;}") as demo:
        with gr.Row():
           
            gr.Markdown(
                f"""
                # LangChain AI Assistant
                Ask questions about LangChain, general topics, or execute Python code.
                """,
                elem_id="app_title" # For CSS styling if needed
            )
        with gr.Accordion("ℹ️ Assistant Details & Controls", open=False):
            gr.Markdown(
                f"""
                - **LLM:** `{LLM_MODEL}`
                - **Embeddings:** `{EMBEDDING_MODEL}`
                - **Available Tools:** `{tools_str}`
                """,
                elem_classes="details_markdown" # For CSS styling if needed
            )
            clear_button = gr.Button("🗑️ Clear Chat History", variant="secondary", size="sm")


        chatbot_display = gr.Chatbot(
            label="Conversation",
            height=500, # Slightly reduced height to accommodate header/footer
            show_copy_button=True,
            bubble_full_width=False,
            render_markdown=True,
            placeholder="Hi there! Ask me anything about LangChain or other topics.",
            elem_id="chatbot_display"
        )

        chat_history_state = gr.State([]) # Manages the conversation history

        with gr.Row(elem_id="input_area"):
            msg_textbox = gr.Textbox(
                show_label=False,
                placeholder="Type your message...",
                container=False,
                lines=1, # Autogrows with input
                # max_lines=5,
                scale=7, # Textbox takes more space
                elem_id="message_textbox"
            )
            send_button = gr.Button("Send", variant="primary", scale=1, elem_id="send_button")


        submit_event_args = {
            "fn": respond,
            "inputs": [msg_textbox, chat_history_state],
            "outputs": [msg_textbox, chatbot_display, chat_history_state],
            "api_name": "send_message" # Optional: for API access
        }

        msg_textbox.submit(**submit_event_args)
        send_button.click(**submit_event_args)

        # Clear button functionality
        def clear_chat_history_action():
            print("[Gradio] Clearing chat history.")
            return "", [], [] # Clear textbox, chatbot display, and history state

        clear_button.click(
            clear_chat_history_action,
            outputs=[msg_textbox, chatbot_display, chat_history_state],
            # queue=False # Making clear immediate
        )

        gr.Examples(
            examples=[
                "What is LangChain Expression Language (LCEL)?",
                "How do I use a TextLoader to load documents?",
                "Show me an example of a custom agent in LangChain.",
                "What is the capital of France?",
                "Calculate 2 to the power of 10",
                "Who won the last FIFA world cup?"
            ],
            inputs=[msg_textbox],
            label="Quick Prompts ✨",
            cache_examples=False # No need to cache simple text inputs
        )

        gr.Markdown(
            """
            <div style='text-align: center; font-size: 0.8em; color: #777; padding-top: 20px;'>
            Powered by LangChain & Gradio.
            </div>
            """,
            elem_id="footer_markdown"
        )

    print("Gradio Blocks interface setup complete.")
    print(f"--- Gradio Setup Complete ---")
    return demo

print("setup_gradio_interface function defined.")

setup_gradio_interface function defined.


In [ ]:
DOCS_DIR = "langchain_docs"
# Gradio Configuration
GRADIO_SERVER_PORT = 7860
GRADIO_SHARE = False
# Cell 8: Main Application Execution Block

print("--- Starting LangChain RAG Chatbot Application ---")

# --- Step 1: Initialize LLM and Embeddings ---
print("\n--- Initializing Models ---")
try:
    # Make llm and embeddings global so functions in other cells can potentially use them
    # Note: It's generally better to pass them as arguments, but following original structure
    global llm, embeddings
    llm = Ollama(model=LLM_MODEL, temperature = 0.1)
    llm.invoke("Respond with just 'LLM OK'") # Quick test
    print(f"LLM Initialized: OK ({LLM_MODEL})")
except Exception as e:
    print(f"ERROR: Failed to initialize Ollama LLM ({LLM_MODEL}): {e}")
    llm = None
    traceback.print_exc()

try:
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
    embeddings.embed_query("Test embedding") # Quick test
    print(f"Embeddings Initialized: OK ({EMBEDDING_MODEL})")
except Exception as e:
    print(f"ERROR: Failed to initialize Ollama Embeddings ({EMBEDDING_MODEL}): {e}")
    embeddings = None
    traceback.print_exc()

# Exit this cell's execution if core components failed
if not llm or not embeddings:
     print("\nFATAL ERROR: Could not initialize LLM or Embeddings. Stopping execution.")
     # In a notebook, just stop here; no need for exit()
     raise RuntimeError("LLM or Embeddings failed to initialize.") # Raise error to stop cell execution

# --- Step 2: Scrape Documents (Conditional) ---
if not os.path.exists(DOCS_DIR) or not any(fname.endswith('.md') for fname in os.listdir(DOCS_DIR)):
    print(f"\nINFO: '{DOCS_DIR}' directory not found or contains no .md files. Running web scraper...")
    try:
         scrape_langchain_docs() # Uses defaults defined in Cell 1
    except Exception as scrape_error:
         print(f"ERROR: Web scraping failed unexpectedly.")
         traceback.print_exc()
         print("WARNING: Proceeding without newly scraped documents due to scraper error.")
else:
    print(f"\nINFO: Found existing document directory: '{DOCS_DIR}' with .md files. Skipping scraping.")

# --- Step 3: Setup Document Retriever (RAG) ---
# Requires embeddings service from Step 1
rag_retriever = None # Ensure it's reset if RAG setup fails
if embeddings: # Only proceed if embeddings initialized
    rag_retriever = setup_document_rag(embedding_service=embeddings) # Calls function from Cell 4
    if rag_retriever:
        print("Document RAG setup check: Successful.")
    else:
        print("Document RAG setup check: Failed or no documents found. Proceeding without RAG tool.")
else:
    print("Skipping RAG setup because embeddings failed to initialize.")


# --- Step 4: Setup Agent Tools ---
# Requires rag_retriever (optional) from Step 3
agent_tools = [] # Ensure it's reset
agent_tools = setup_tools(retriever=rag_retriever, serpapi_key=SERPAPI_API_KEY) # Calls function from Cell 5
if not agent_tools:
     print("WARNING: No agent tools were configured. Agent capabilities will be limited.")

# --- Step 5: Initialize Conversational Agent ---
# Requires llm from Step 1 and agent_tools from Step 4
agent_executor = None # Ensure it's reset
if llm: # Only proceed if LLM initialized
    agent_executor = initialize_conversational_agent(tools_list=agent_tools, agent_llm=llm) # Calls function from Cell 6
    if not agent_executor:
         print("ERROR: Failed to initialize the conversational agent. UI will show an error message.")
else:
    print("Skipping agent initialization because LLM failed to initialize.")

# --- Step 6: Setup Gradio Interface ---
# Requires agent_executor from Step 5
gradio_app = setup_gradio_interface(agent_executor, agent_tools_list=agent_tools) # Calls function from Cell 7

# --- Step 7: Launch Gradio Application ---
if gradio_app:
    print("\n--- Launching Gradio App ---")
    try:
        print(f"Attempting to launch Gradio on: http://127.0.0.1:{GRADIO_SERVER_PORT}")
        # Launch in a way that works well in notebooks (inline or new tab)
        # share=True generates a public link (requires Gradio account/tunneling)
        gradio_app.launch(
            server_name="127.0.0.1",
            server_port=GRADIO_SERVER_PORT,
            share=GRADIO_SHARE,
            # In a notebook, you might not need blocking, but launch() usually handles this.
            # prevent_thread_lock=True, # May help in some notebook environments
        )
        # Note: In some notebook setups, you might need to manually stop the kernel
        # to fully shut down the Gradio server after you're done.
    except OSError as oe:
        if "address already in use" in str(oe).lower() or "cannot find empty port" in str(oe).lower():
             print(f"\n*** GRADIO ERROR: Port {GRADIO_SERVER_PORT} is already in use! ***")
             print(f"Stop any other running Gradio apps or change GRADIO_SERVER_PORT in Cell 1.")
        else:
             print(f"ERROR: Failed to launch Gradio app due to an OS Error:")
             traceback.print_exc()
    except Exception as e:
        print(f"ERROR: Failed to launch Gradio app:")
        traceback.print_exc()
else:
    print("\n--- Gradio App Launch Skipped ---")
    print("Gradio app could not be launched because the UI setup failed (likely due to agent initialization failure). Check previous logs.")

print("\n--- Application Setup and Launch Attempt Finished ---")

--- Starting LangChain RAG Chatbot Application ---

--- Initializing Models ---
LLM Initialized: OK (llama3:8b)
Embeddings Initialized: OK (nomic-embed-text)

INFO: Found existing document directory: 'langchain_docs' with .md files. Skipping scraping.

--- Setting up Document RAG ---
Document Source: langchain_docs
FAISS Index Path: faiss_langchain_docs_nomic_index
Attempting to load existing FAISS index from: faiss_langchain_docs_nomic_index
FAISS index loaded successfully.
--- Document RAG Setup Complete (Loaded Existing Index) ---
Document RAG setup check: Successful.

--- Setting up Agent Tools ---
  Tool Added: LangChain Document Search
  Tool Added: Web Search (SerpAPI)
  Tool Added: Python REPL
Initialized Tools: ['LangChain Document Search', 'Web Search', 'Python REPL']
--- Finished Setting up Agent Tools ---

--- Initializing Conversational Agent ---
Initializing ConversationTokenBufferMemory (max_token_limit=3500)...
Memory initialized.
Initializing agent with AgentType: Agen

C:\Users\Radwa\AppData\Local\Temp\ipykernel_23208\1915829423.py:139: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_display = gr.Chatbot(
C:\Users\Radwa\AppData\Local\Temp\ipykernel_23208\1915829423.py:139: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_display = gr.Chatbot(


Gradio Blocks interface setup complete.
--- Gradio Setup Complete ---

--- Launching Gradio App ---
Attempting to launch Gradio on: http://127.0.0.1:7860
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



--- Application Setup and Launch Attempt Finished ---



[Gradio] User Input: 'who is the president of the USA?'
[Gradio] Invoking agent executor...


> Entering new AgentExecutor chain...
```json
{
    "action": "Web Search",
    "action_input": "Who is the current President of the United States?"
}
```

Please note that I've used the Web Search tool as the user's input doesn't seem to be related to LangChain features or concepts. If you'd like me to use a different tool, please let me know!  [Tool Call] Web Search: 'Who is the current President of the United States?...'
    -> Web search result received.

Observation: Donald Trump
Thought:Here's the response:

```json
{
    "action": "Final Answer",
    "action_input": "As of my knowledge cutoff, the President of the United States is Donald Trump."
}
```

> Finished chain.
[Gradio] Agent Raw Output (first 2000 chars): As of my knowledge cutoff, the President of the United States is Donald Trump....
[Gradio] Agent response seems valid.
[Gradio] Responding to UI...
